In [1]:
import tensorflow

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

In [18]:
batch_size = 64
epochs = 10
latent_dim = 256
num_samples = 20000

In [19]:
data_path = '/content/drive/MyDrive/fra.txt'

In [20]:
input_texts = []
output_texts = []
input_characters = set()
output_characters = set()

with open(data_path, 'r', encoding = 'utf-8') as f:
    lines = f.read().split('\n')
for line in lines[:min(num_samples,len(lines)-1)]:
    input_text, output_text, _ = line.split('\t')
    
    # We use 'tab' as the 'start sequence' character
    # for the targets, and '\n' as the 'end sequence' character.
    
    output_text = '\t' + output_text + '\n'
    input_texts.append(input_text)
    output_texts.append(output_text)

    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in output_text:
        if char not in output_characters:
            output_characters.add(char)

In [21]:
len(output_characters)

100

In [22]:
input_characters = sorted(list(input_characters))
output_characters = sorted(list(output_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(output_characters)
max_encoder_seq_length = max([len(text) for text in input_texts])
max_decoder_seq_length = max([len(text) for text in output_texts])

In [23]:
print("Number of Samples:", len(input_texts))
print('Number of unique input Tokens:' , num_encoder_tokens)
print('Number of unique output Tokens:' , num_decoder_tokens)
print('Max sequence length for inputs:' , max_encoder_seq_length)
print('Max sequence length for outputs:' , max_decoder_seq_length)

Number of Samples: 20000
Number of unique input Tokens: 74
Number of unique output Tokens: 100
Max sequence length for inputs: 17
Max sequence length for outputs: 59


In [24]:
input_token_index = dict(
    [(char,i) for i,char in enumerate(input_characters)])

output_token_index = dict(
    [(char,i) for i,char in enumerate(output_characters)])

In [25]:
input_token_index , output_token_index

({' ': 0,
  '!': 1,
  '"': 2,
  '$': 3,
  '%': 4,
  '&': 5,
  "'": 6,
  ',': 7,
  '-': 8,
  '.': 9,
  '/': 10,
  '0': 11,
  '1': 12,
  '2': 13,
  '3': 14,
  '4': 15,
  '5': 16,
  '6': 17,
  '7': 18,
  '8': 19,
  '9': 20,
  ':': 21,
  '?': 22,
  'A': 23,
  'B': 24,
  'C': 25,
  'D': 26,
  'E': 27,
  'F': 28,
  'G': 29,
  'H': 30,
  'I': 31,
  'J': 32,
  'K': 33,
  'L': 34,
  'M': 35,
  'N': 36,
  'O': 37,
  'P': 38,
  'Q': 39,
  'R': 40,
  'S': 41,
  'T': 42,
  'U': 43,
  'V': 44,
  'W': 45,
  'Y': 46,
  'a': 47,
  'b': 48,
  'c': 49,
  'd': 50,
  'e': 51,
  'f': 52,
  'g': 53,
  'h': 54,
  'i': 55,
  'j': 56,
  'k': 57,
  'l': 58,
  'm': 59,
  'n': 60,
  'o': 61,
  'p': 62,
  'q': 63,
  'r': 64,
  's': 65,
  't': 66,
  'u': 67,
  'v': 68,
  'w': 69,
  'x': 70,
  'y': 71,
  'z': 72,
  'é': 73},
 {'\t': 0,
  '\n': 1,
  ' ': 2,
  '!': 3,
  '$': 4,
  '%': 5,
  '&': 6,
  "'": 7,
  '(': 8,
  ')': 9,
  ',': 10,
  '-': 11,
  '.': 12,
  '0': 13,
  '1': 14,
  '2': 15,
  '3': 16,
  '4': 17,
  '5'

In [26]:
encoder_input_data = np.zeros(
    (len(input_texts),max_encoder_seq_length,num_encoder_tokens),dtype = 'float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length , num_decoder_tokens), dtype = 'float32')
decoder_output_data = np.zeros(
    (len(input_texts), max_decoder_seq_length,num_decoder_tokens) , dtype = 'float32')

In [27]:
for i,(input_text,output_text) in enumerate(zip(input_texts,output_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i,t,input_token_index[char]] = 1.
    encoder_input_data[i,t+1:,input_token_index[' ']] = 1.
    
    for t, char in enumerate(output_text):
        # decoder_output_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i,t,output_token_index[char]] = 1.
        if t>0:
            # decoder_output_data will be ahead by one timestep
            # and will not include the start character
            decoder_output_data[i,t-1,output_token_index[char]] = 1.
    decoder_input_data[i,t+1:, output_token_index[' ']] = 1.
    decoder_output_data[i , t: , output_token_index[' ']] = 1.

In [28]:
encoder_input_data[0].shape

(17, 74)

In [29]:
# define an input sequence and process it:
encoder_inputs = Input(shape = (None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state= True)
encoder_outputs,state_h,state_c = encoder(encoder_inputs)
# we discard 'encoder_outputs' and only keep the states.
encoder_states = [state_h,state_c]

In [30]:
# set up the decoder , using encoder_states as initial states:
decoder_inputs= Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim,return_sequences = True, return_state = True)
decoder_outputs, _ ,_ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens,activation = 'softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [31]:
# Define the model that will turn
# 'encoder_input_data' & 'decoder_input_data' into 'decoder_output_data'
model = Model([encoder_inputs,decoder_inputs], decoder_outputs)

#Run training:
model.compile(optimizer='rmsprop', loss = 'categorical_crossentropy' , metrics = ['accuracy'])
model.fit([encoder_input_data,decoder_input_data], decoder_output_data,
         batch_size=batch_size,
         epochs = epochs,
         validation_split=0.2)

Epoch 1/10
250/250 [==============================] - 101s 391ms/step - loss: 1.0367 - accuracy: 0.7381 - val_loss: 0.9075 - val_accuracy: 0.7461
Epoch 2/10
250/250 [==============================] - 97s 387ms/step - loss: 0.6846 - accuracy: 0.8041 - val_loss: 0.7267 - val_accuracy: 0.7890
Epoch 3/10
250/250 [==============================] - 99s 395ms/step - loss: 0.5785 - accuracy: 0.8301 - val_loss: 0.6511 - val_accuracy: 0.8098
Epoch 4/10
250/250 [==============================] - 97s 390ms/step - loss: 0.5213 - accuracy: 0.8457 - val_loss: 0.6090 - val_accuracy: 0.8198
Epoch 5/10
250/250 [==============================] - 98s 391ms/step - loss: 0.4810 - accuracy: 0.8564 - val_loss: 0.5709 - val_accuracy: 0.8314
Epoch 6/10
250/250 [==============================] - 96s 386ms/step - loss: 0.4492 - accuracy: 0.8654 - val_loss: 0.5467 - val_accuracy: 0.8373
Epoch 7/10
250/250 [==============================] - 98s 392ms/step - loss: 0.4229 - accuracy: 0.8729 - val_loss: 0.5272 - val_a

In [32]:
# define sampling models:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape = (latent_dim,))
decoder_state_input_c = Input(shape = (latent_dim,))
decoder_input_states = [decoder_state_input_h,decoder_state_input_c]

decoder_outputs,state_h,state_c = decoder_lstm(decoder_inputs, initial_state=decoder_input_states)
decoder_states = [state_h,state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs]+decoder_input_states,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_index = dict(
    (i,char) for char,i in input_token_index.items())
reverse_output_char_index = dict(
    (i,char) for char,i in output_token_index.items())

def decode_sequence(input_seq):
    states_value= encoder_model.predict(input_seq)
    output_seq = np.zeros((1,1,num_decoder_tokens))
    output_seq[0,0,output_token_index['\t']] = 1
    
    stop_condition = False
    decoded_sentences = ''
    while not stop_condition:
        output_tokens,h,c = decoder_model.predict(
            [output_seq]+ states_value)
        
        sampled_token_index = np.argmax(output_tokens[0,-1, :])
        sampled_char = reverse_output_char_index[sampled_token_index]
        decoded_sentences += sampled_char

        if(sampled_char == '\n' or len(decoded_sentences) > max_decoder_seq_length):
            stop_condition = True
        
        #update the target sequence (of length 1):
        output_seq = np.zeros((1,1,num_decoder_tokens))
        output_seq[0,0,sampled_token_index] = 1
        
        states_value = [h,c]
    return decoded_sentences

for seq_index in range(20):
    # take one sequence for trying out decoding:
    input_seq = encoder_input_data[seq_index:seq_index+1]
    decoded_sentences = decode_sequence(input_seq)
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:' , decoded_sentences)

1/1 [==============================] - 0s 20ms/step
Input sentence: Go.
Decoded sentence: Déculez-vous !

1/1 [==============================] - 0s 24ms/step
Input sentence: Go.
Decoded sentence: Déculez-vous !

1/1 [==============================] - 0s 19ms/step
Input sentence: Go.
Decoded sentence: Déculez-vous !

1/1 [==============================] - 0s 20ms/step
Input sentence: Go.
Decoded sentence: Déculez-vous !

1/1 [==============================] - 0s 23ms/step
Input sentence: Hi.
Decoded sentence: Attendez !

1/1 [==============================] - 0s 22ms/step
Input sentence: Hi.
Decoded sentence: Attendez !

1/1 [==============================] - 0s 22ms/step
Input sentence: Run!
Decoded sentence: Prenez le compe !

1/1 [==============================] - 0s 22ms/step
Input sentence: Run!
Decoded sentence: Prenez le compe !

1/1 [==============================] - 0s 20ms/step
Input sentence: Run!
Decoded sentence: Prenez le compe !

1/1 [==============================] - 0s 